In [1]:
import numpy as np
# import pandas
import pandas as pd

# import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
import string

In [2]:
data = '/Users/patrickokwir/Desktop/Git_Projects/Ted-Talks-Recommender-System/Data_output/talks_2.csv'
df = pd.read_csv(data, index_col=0)
df.head()

,author,talk,description,likes,views,transcript
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio..."
1,Fabio Pacucci,the battle that formed the universe,its time for the biggest battle in the univers...,51,Fabio,Welcome one and all! It’s time to grab your se...
2,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...
3,Rebecca Darwent,how to fund real change in your community,is there a way to give back that benefits ever...,1000,341218,I spent my whole career in the nonprofit secto...
4,Susanne Buckley-Zistel,what caused the rwandan genocide,for one hundred days in 1994 the african count...,3700,126376,"For 100 days in 1994, the African country of R..."


In [3]:
# drop rows where df['transcript'] is null
df = df[df['transcript'].notnull()]

In [4]:
df['transcript'].isnull().sum()

0

In [5]:
# --------------------------------------------------Imports--------------------------------------------------#
import re
import nltk
import spacy
import gensim
import pyLDAvis
import nlp
nltk.download('stopwords')


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

#--------------------------------------------------Things to Note--------------------------------------------------#
#The following are key factors to obtaining good segregation topics:**

#The quality of text processing.
#The variety of topics the text talks about.
#The choice of topic modeling algorithm.
#The number of topics fed to the algorithm.
#The algorithms tuning parameters.

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/patrickokwir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#___________________________________________________Data Processing__________________________________________________#

# Convert to list
df['transcript'].astype(str)
data = df['transcript'].values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', i) for i in data]

# Remove new line characters in data
data = [re.sub('\s+', ' ', i) for i in data]

# Remove distracting single quotes
data = [re.sub("\'", "", i) for i in data]

# import pprint
from pprint import pprint

<>:8: DeprecationWarning: invalid escape sequence \S
<>:11: DeprecationWarning: invalid escape sequence \s
<>:8: DeprecationWarning: invalid escape sequence \S
<>:11: DeprecationWarning: invalid escape sequence \s
/var/folders/7f/4z7lvktj44g121hm_1s6v18h0000gn/T/ipykernel_73556/1779527103.py:8: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', i) for i in data]
/var/folders/7f/4z7lvktj44g121hm_1s6v18h0000gn/T/ipykernel_73556/1779527103.py:11: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', i) for i in data]


In [7]:
#break down each sentence into a list of words through tokenization
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True, min_len=4))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['hello', 'machine', 'dazzle', 'emotionally', 'driven', 'instinct', 'based', 'conceptual', 'artist', 'designer', 'musician', 'today', 'apos', 'going', 'talk', 'about', 'power', 'transformation', 'through', 'sacred', 'ritual', 'costume', 'with', 'help', 'friend', 'matty', 'crosland', 'matty', 'girl', 'laughs', 'matty', 'going', 'hand', 'seeds', 'possibility', 'while', 'continue', 'growing', 'middle', 'america', 'wasn', 'apos', 'allowed', 'discouraged', 'from', 'career', 'arts', 'when', 'went', 'store', 'brothers', 'would', 'black', 'blue', 'aisle', 'would', 'pink', 'aisle', 'there', 'shame', 'surrounding', 'preference', 'quot', 'girls', 'apos', 'quot', 'toys', 'shoplifted', 'first', 'barbie', 'desperate', 'people', 'desperate', 'things', 'what', 'really', 'happening', 'during', 'that', 'time', 'building', 'massive', 'reserve', 'ideas', 'unrealized', 'desires', 'dreams', 'when', 'bought', 'that', 'ticket', 'york', 'which', 'first', 'decision', 'ever', 'truly', 'made', 'myself', 'brought

In [8]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

In [9]:
from nltk.corpus import stopwords


stop_words = stopwords.words('english')
stop_words.extend(['music', 'stephen', 'detail', 'way', 'shares', 'need', 'us', 'come','order','try','go','get','make','drink','plate','dish','restaurant','place', 'many', 'day', 'explains', 'even', 'part',
                  'would','really','like','great','service','came','got', 'talk', 'directed', 'ted', 'narrated', 'new', 'one', 'using', 'addison', 'anderson', 'says', "addison_anderson", "years", "first", 
                  'maybe', 'someone', 'pserson', 'applause', 'laughter', 'cheers', 'know', 'actually', 'worlds', 'could', 'details', 'studio', 'help', 'music', 'life', 'shows', 'world', 'good', 'think', 
                  'story', 'stories', 'wanted', 'moment', 'yeah', 'sound', 'night', 'guys', 'stuff', 'video', 'okay', 'everybody'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def bigrams(words, bi_min=12, tri_min=7):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

# def get_corpus(df):
#     df['text'] = df['review']
#     df['text'] = strip_newline(df.text)
#     words = list(sent_to_words(df.text))
#     words = remove_stopwords(words)
#     bigram_mod = bigrams(words)
#     bigram = [bigram_mod[review] for review in words]
#     id2word = gensim.corpora.Dictionary(bigram)
#     id2word.filter_extremes(no_below=10, no_above=0.35)
#     id2word.compactify()
#     corpus = [id2word.doc2bow(text) for text in bigram]
#     return corpus, id2word, bigram

In [10]:
words = remove_stopwords(data_words)
bigram = bigrams(words)
bigram = [bigram[review] for review in words]
id2word = gensim.corpora.Dictionary(bigram)
id2word.filter_extremes(no_below=15, no_above=0.2)
id2word.compactify()
corpus = [id2word.doc2bow(text) for text in bigram]


In [11]:
train_corpus4 = corpus
train_id2word4 = id2word
bigram_train4 = bigram

In [12]:
import warnings

In [13]:
# import logging
# logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus4,
                           num_topics=10,
                           id2word=train_id2word4,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 10,
                           per_word_topics=True)
    # lda_train4.save('lda_train4.model')

In [14]:
coherence_model_lda = CoherenceModel(model=lda_train4, texts=bigram_train4, dictionary=train_id2word4, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
coherence_lda

0.5338261696875537

In [15]:
lda_train4.print_topics(13,num_words=5)[:13]

[(0,
  '0.007*"government" + 0.006*"political" + 0.005*"united_states" + 0.005*"violence" + 0.004*"police"'),
 (1,
  '0.006*"brain" + 0.006*"light" + 0.005*"design" + 0.005*"computer" + 0.005*"universe"'),
 (2,
  '0.014*"species" + 0.013*"animals" + 0.009*"fish" + 0.009*"ocean" + 0.006*"animal"'),
 (3,
  '0.009*"brain" + 0.006*"child" + 0.005*"sleep" + 0.005*"pain" + 0.005*"care"'),
 (4,
  '0.009*"language" + 0.006*"century" + 0.005*"chinese" + 0.005*"africa" + 0.004*"city"'),
 (5,
  '0.012*"cells" + 0.009*"disease" + 0.006*"cancer" + 0.006*"cell" + 0.006*"virus"'),
 (6,
  '0.005*"kids" + 0.003*"woman" + 0.003*"book" + 0.003*"house" + 0.003*"mother"'),
 (7,
  '0.007*"countries" + 0.006*"companies" + 0.005*"company" + 0.005*"business" + 0.004*"global"'),
 (8,
  '0.006*"students" + 0.004*"kids" + 0.003*"education" + 0.003*"social" + 0.003*"ideas"'),
 (9,
  '0.011*"energy" + 0.009*"food" + 0.008*"city" + 0.007*"cities" + 0.006*"planet"')]

In [16]:
# exract topics from the model
topics = lda_train4.show_topics(formatted=False)
for topic in topics:
    print("Topic", topic[0])
    print("Words:", [word[0] for word in topic[1]])
    print("Weights:", [word[1] for word in topic[1]])
    print("\n")

Topic 0
Words: ['government', 'political', 'united_states', 'violence', 'police', 'democracy', 'america', 'black', 'state', 'vote']
Weights: [0.006578415, 0.0059280964, 0.004946293, 0.004938919, 0.004397281, 0.0043818234, 0.0042502466, 0.0039019368, 0.0035865835, 0.0033268244]


Topic 1
Words: ['brain', 'light', 'design', 'computer', 'universe', 'machine', 'image', 'robot', 'science', 'robots']
Weights: [0.006399746, 0.0063120914, 0.0053746616, 0.0049595973, 0.004904079, 0.0036539102, 0.0030246382, 0.0030177394, 0.0026315732, 0.0026222595]


Topic 2
Words: ['species', 'animals', 'fish', 'ocean', 'animal', 'humans', 'nature', 'planet', 'birds', 'deep']
Weights: [0.014238683, 0.013056246, 0.009259692, 0.009168176, 0.0064678444, 0.0047873394, 0.00469341, 0.0045390734, 0.004152414, 0.003574822]


Topic 3
Words: ['brain', 'child', 'sleep', 'pain', 'care', 'patients', 'health', 'hospital', 'parents', 'mother']
Weights: [0.008504208, 0.005941857, 0.0052454723, 0.0050987634, 0.0050604166, 0.00

In [17]:
# function to text number of topics to make 
topics = (8, 9, 10, 11, 12, 13)
results = {}
def test_num_topics(num_topics):
    for topic in topics: 
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                                corpus=train_corpus4,
                                num_topics=topic,
                                id2word=train_id2word4,
                                chunksize=100,
                                workers=7, # Num. Processing Cores - 1
                                passes=50,
                                eval_every = 10,
                                per_word_topics=True)
            coherence_model_lda = CoherenceModel(model=lda_train4, texts=bigram_train4, dictionary=train_id2word4, coherence='c_v')
            coherence_lda = coherence_model_lda.get_coherence()
            results.update({topic: coherence_lda})

In [18]:
test_num_topics(topics)

In [ ]:
import nltk

# Download necessary resources for tokenization
nltk.download('punkt')

# Assuming you have already trained your LDA model and have it stored in the 'lda_model' variable

# Create a list to store the extracted topics
extracted_topics = []

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    # Extract the text from the appropriate column in your dataframe
    text1 = row['transcript']
    text = clean_text(text1)
    
    # Tokenize the text into individual tokens
    tokens = nltk.word_tokenize(text)
    
    # Preprocess the tokens if necessary (e.g., removing stopwords, stemming, etc.)
    
    # Convert the tokens to a bag-of-words representation
    bow_vector = lda_train4.id2word.doc2bow(tokens)
    
    # Get the topic distribution for the document
    topic_distribution = lda_train4.get_document_topics(bow_vector)
    
    # Sort the topics by their probability in descending order
    sorted_topics = sorted(topic_distribution, key=lambda x: x[1], reverse=True)
    
    # Extract the most probable topic (assuming the topics are represented as tuples (topic_id, probability))
    most_probable_topic = sorted_topics[0][0]
    if most_probable_topic == 0:
        most_probable_topic = topics[0][1][0][0], topics[0][1][1][0], topics[0][1][2][0], topics[0][1][3][0]
    elif most_probable_topic == 1:
        most_probable_topic = topics[1][1][0][0], topics[1][1][1][0], topics[1][1][2][0], topics[1][1][3][0]
    elif most_probable_topic == 2:
        most_probable_topic = topics[2][1][0][0], topics[2][1][1][0], topics[2][1][2][0], topics[2][1][3][0]
    elif most_probable_topic == 3:
        most_probable_topic = topics[3][1][0][0], topics[3][1][1][0], topics[3][1][2][0], topics[3][1][3][0]
    elif most_probable_topic == 4:
        most_probable_topic = topics[4][1][0][0], topics[4][1][1][0], topics[4][1][2][0], topics[4][1][3][0]
    elif most_probable_topic == 5:
        most_probable_topic = topics[5][1][0][0], topics[5][1][1][0], topics[5][1][2][0], topics[5][1][3][0]
    elif most_probable_topic == 6:
        most_probable_topic = topics[6][1][0][0], topics[6][1][1][0], topics[6][1][2][0], topics[6][1][3][0]
    elif most_probable_topic == 7:
        most_probable_topic = topics[7][1][0][0], topics[7][1][1][0], topics[7][1][2][0], topics[7][1][3][0]
    # elif most_probable_topic == 8:
    #     most_probable_topic = topics[8][1][0][0], topics[8][1][1][0], topics[8][1][2][0], topics[8][1][3][0]
    # elif most_probable_topic == 9:
    #     most_probable_topic = topics[9][1][0][0], topics[9][1][1][0], topics[9][1][2][0], topics[9][1][3][0]
    # Append the most probable topic to the extracted_topics list
    extracted_topics.append(most_probable_topic)
df['Tags'] = extracted_topics

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/patrickokwir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df.head()

,author,talk,description,likes,views,transcript,Tags
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio...","(city, design, project, death)"
1,Fabio Pacucci,the battle that formed the universe,its time for the biggest battle in the univers...,51,Fabio,Welcome one and all! It’s time to grab your se...,"(light, universe, energy, planet)"
2,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...,9
3,Rebecca Darwent,how to fund real change in your community,is there a way to give back that benefits ever...,1000,341218,I spent my whole career in the nonprofit secto...,"(countries, companies, company, business)"
4,Susanne Buckley-Zistel,what caused the rwandan genocide,for one hundred days in 1994 the african count...,3700,126376,"For 100 days in 1994, the African country of R...","(government, political, black, united_states)"


In [ ]:
df[df['author'] == 'Simon Sinek'].values

array([['Simon Sinek', 'how to discover your why in difficult times',
        'what has the coronavirus pandemic taught us about ourselves and our relationships in a deeply personal and wideranging conversation leadership expert simon sinek shares his own experience caring for his mental health as the world shut down he discusses why we need to nurture friendships in both good times and bad explains why anyone can be a leader  and reveals the secret to discovering your why in life this virtual conversation hosted by head of ted chris anderson was part of an exclusive ted membership event visit tedcommembership to become a ted member',
        '6500', '2179503',
        'Chris Anderson: Simon, I&apos;ll start us off by saying, I mean, here we are, look, after a year of the pandemic, probably one of the most extraordinary experiences any of us have had. What do you think the unexpected psychological carryovers might be? I mean, do you think we&apos;ve kind of -- Part of me thinks that pe